### Graph Neural Network (GNN)

$$h_{i}^{t+1} = f\left(h_{i}^{t} W + \sum_{j \in N (i)} \frac{1}{C_{i,j}} h_{j}^{t} U \right)$$

Old representation times a weight matrix:
$$h_{i}^{t} W $$

Information from neighbors times a weight matrix:
$$h_{j}^{t} U $$

### Aggregation function:

Sum over all transformed neighbor representations

$$\sum_{j \in N (i)} \frac{1}{C_{i,j}}$$

Normalize the vectors differently for each neighbor
$$\frac{1}{C_{i,j}}$$

The sum is a $permutation-invariant$ aggregation function -> Insensitive to $order$

Each node's updated value becomes a weighting of its previous value+weightning of it's neighbors values.

-> Agg function can be mean, max, concatenation, etc.



Collapse $W_{self}$ and $W_{neigh}$ into $W$ by adding self-loops to the adjacency matrix $A$:
$$H^{(k+1)} = \sigma \left( (A+I)H^{(k)} W^{k+1} \right)$$

In [120]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, DataParallel, Linear
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch_geometric.transforms as T

import csv
import seaborn as sb
import matplotlib.pyplot as plt

### Graph data
<div style="text-align:center; display: flex; justify-content: center;">
  <table>
    <tr>
      <th>Rosette number</th>
      <th>Nodes</th>
      <th>Edges</th>
    </tr>
    <tr>
      <td>3</td>
      <td>11157</td>
      <td>7572</td>
    </tr>
    <tr>
      <td>6</td>
      <td>9568</td>
      <td>5245</td>
    </tr>
    <tr>
      <td>7</td>
      <td>11635</td>
      <td>9257</td>
    </tr>
    <tr>
      <td>11</td>
      <td>13667</td>
      <td>13051</td>
    </tr>
    <tr>
      <td>12</td>
      <td>10617</td>
      <td>6870</td>
    </tr>
    <tr>
      <td>13</td>
      <td>13260</td>
      <td>14395</td>
    </tr>
    <tr>
      <td>14</td>
      <td>10704</td>
      <td>7635</td>
    </tr>
    <tr>
      <td>15</td>
      <td>10131</td>
      <td>8655</td>
    </tr>
    <tr>
      <td>18</td>
      <td>11117</td>
      <td>7991</td>
    </tr>
    <tr>
      <td>19</td>
      <td>10248</td>
      <td>6689</td>
    </tr>
  </table>
</div>



In [99]:
def graph_r(r):

    nodes = []
    edges = []
    mass = []

    with open(f'./data/rosette{r}_nodes.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            if (row!=0):
                values = list(row.values())
                n = []
                n.append(float(values[0]))
                n.extend(22.5-2.5*np.log10([float(n) for n in values[2:]]))
                n.append(float(values[-1]))
                nodes.append(n)
                mass.append(float(values[2]))

    with open(f'./data/rosette{r}_edges.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            if (row!=0):
                edges.append([float(n) for n in list(row.values())])

    return (nodes,edges,mass)

In [71]:
rosettes = [3,6,7,11,12,13,14,15,18,19]

### GNN Model

In [49]:
class GNN(nn.Module):
    def __init__(self, task='node'):
        super(GNN, self).__init__()
        torch.manual_seed(26)

        self.init_conv = GCNConv(graph.num_node_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size) #GAT or GCN?
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)
        self.out = Linear(embedding_size*2, 1) #Regression problem

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        hidden = self.init_conv(x, edge_index)
        hidden = nn.ReLU(hidden)

        hidden = self.conv1(hidden, edge_index)
        hidden = nn.ReLU(hidden)

        hidden = self.conv2(hidden, edge_index)
        hidden = nn.ReLU(hidden)

        hidden = self.conv3(hidden, edge_index)
        hidden = nn.ReLU(hidden)

        #Global pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), gap(hidden, batch_index)], dim=1)
        out = self.out(hidden)

        return out, hidden

In [150]:
model = GNN()

embedding_size = 64
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GNN(
  (init_conv): GCNConv(8, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(128, 1, bias=True)
)
Number of parameters:  13185


In [64]:
optimizer = SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [65]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [148]:
nodes, edges, mass = graph_r(rosettes[0])

nodes_train, nodes_test, mass_train, mass_test = train_test_split(nodes, mass, test_size=0.8, random_state=26)

nodes_train, nodes_test = torch.tensor(nodes_train, dtype=torch.float), torch.tensor(nodes_test, dtype=torch.float)
mass_train, mass_test = torch.tensor(mass_train, dtype=torch.float), torch.tensor(mass_test, dtype=torch.float)

edges = torch.tensor(edges, dtype=torch.long)
edge_index = edges[:, :2].t().contiguous()

graph_train = Data(x=nodes_train, edge_index=edge_index, edge_attr=edges[:, 2], y=mass_train)
graph_test = Data(x=nodes_test, edge_index=edge_index, edge_attr=edges[:, 2], y=mass_test)

graph_train.num_nodes = nodes_train.size(0)
graph_test.num_nodes = nodes_test.size(0)
graph_train.num_edges, graph_test.num_edges = edges.size(0), edges.size(0)

In [149]:
graphs_per_batch = 64
loader = DataLoader(graph_train, batch_size=graphs_per_batch, shuffle=True)
test_loader = DataLoader(graph_test, batch_size=graphs_per_batch, shuffle=True)

### Training

In [147]:
def train(data):
    for batch in loader:
      batch.to(device) #gpu
      optimizer.zero_grad() #reset gradients
      pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch) #passing node features and connection info
      loss = loss_fn(pred, batch.y) #calculate loss and gradients
      loss.backward()
      optimizer.step() #update using gradients
    return loss, embedding

number_epochs = 100
losses = []
for epoch in range(number_epochs):
    loss, h = train(graph)
    losses.append(loss)
    if epoch % 100 == 0:
      print(f"Epoch: {epoch} | Train Loss: {loss}")

Starting training...


KeyError: 0

In [ ]:
losses_float = [float(loss.cpu().detach().numpy()) for loss in losses]
loss_indices = [i for i,l in enumerate(losses_float)]
plt = sns.lineplot(loss_indices, losses_float) #plot training loss
plt